```{contents}
```
## **Human Node in LangGraph**

A **Human Node** in LangGraph is a specialized node that **pauses autonomous execution** and explicitly transfers control to a human operator before continuing the workflow.
It enables **human-in-the-loop AI systems** where safety, correctness, or compliance cannot be fully automated.

---

### **1. Why Human Nodes Exist**

Purely autonomous LLM systems are unsafe for:

* High-risk decisions (finance, medicine, legal)
* Irreversible actions (deployments, payments, deletions)
* Ambiguous tasks requiring judgment
* Regulatory compliance

Human nodes provide a **control boundary** between automation and accountability.

---

### **2. Conceptual Model**

```
[ Autonomous Processing ]
          ↓
     [ Human Node ]
          ↓
[ Resume Autonomous Processing ]
```

At the human node, execution **blocks** until human input modifies the state.

---

### **3. Core Responsibilities of a Human Node**

| Responsibility     | Function                            |
| ------------------ | ----------------------------------- |
| Pause execution    | Stop graph until resolved           |
| Expose state       | Show current state to user          |
| Collect feedback   | Receive edits, approvals, decisions |
| Inject corrections | Modify state before continuing      |
| Record audit trail | Persist human actions               |

---

### **4. How Human Nodes Work Internally**

Human nodes are implemented using **interrupt semantics**:

1. Graph reaches human node
2. Runtime emits an **interrupt event**
3. Current state is **checkpointed**
4. UI or API presents state to human
5. Human submits updates
6. Graph **resumes** from same node

---

### **5. Basic Implementation Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str
    approved: bool

def draft(state):
    return {"text": "Draft response", "approved": False}

def human_review(state):
    # Execution pauses here in real runtime
    print("Waiting for human approval...")
    return state

def finalize(state):
    return {"text": state["text"] + " [FINAL]"}

builder = StateGraph(State)

builder.add_node("draft", draft)
builder.add_node("review", human_review)
builder.add_node("final", finalize)

builder.set_entry_point("draft")
builder.add_edge("draft", "review")
builder.add_conditional_edges("review", lambda s: "final" if s["approved"] else END, {
    "final": "final",
    END: END
})

graph = builder.compile()
```

---

### **6. Real Human-in-the-Loop Execution Pattern**

In production, the `human_review` node triggers an interrupt:

```python
result = graph.invoke(input, config={"interrupt_before": ["review"]})
```

The system returns:

```json
{
  "state": { "text": "Draft response", "approved": false },
  "next": "review"
}
```

After human updates:

```python
graph.invoke(
    {"text": "Edited by human", "approved": True},
    config={"resume": True}
)
```

---

### **7. Common Variants of Human Nodes**

| Variant               | Purpose                  |
| --------------------- | ------------------------ |
| Approval Gate         | Yes/No decision          |
| Review & Edit         | Modify generated content |
| Correction Node       | Fix errors               |
| Override Node         | Replace model output     |
| Escalation Node       | Send to expert           |
| Multi-Human Consensus | Multiple reviewers       |

---

### **8. Safety & Governance Benefits**

| Feature        | Benefit                   |
| -------------- | ------------------------- |
| Interrupt      | Prevents unsafe execution |
| Checkpointing  | No data loss              |
| Audit logging  | Regulatory compliance     |
| Rollback       | Undo actions              |
| Accountability | Human ownership           |

---

### **9. Where Human Nodes Are Used**

* Medical decision support
* Financial transaction approval
* Legal document generation
* Deployment automation
* Autonomous agents with kill-switch
* Content moderation systems

---

### **10. Mental Model**

> A Human Node converts an LLM system from a **black box** into a **supervised control system**.

It enables **trustworthy, compliant, and deployable AI**.

### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    draft: str
    approved: bool

# Step 1: LLM / system generates a draft
def create_draft(state):
    return {
        "draft": "This is an auto-generated response.",
        "approved": False
    }

# Step 2: Human node
def human_review(state):
    # Real execution will pause here
    return state

# Step 3: Finalization
def finalize(state):
    return {"draft": state["draft"] + " [FINAL]"}

builder = StateGraph(State)

builder.add_node("draft", create_draft)
builder.add_node("review", human_review)
builder.add_node("final", finalize)

builder.set_entry_point("draft")
builder.add_edge("draft", "review")

builder.add_conditional_edges(
    "review",
    lambda s: "final" if s["approved"] else END,
    {"final": "final", END: END}
)

graph = builder.compile()


In [2]:
result = graph.invoke({}, config={"interrupt_before": ["review"]})


In [3]:
result

{'draft': 'This is an auto-generated response.', 'approved': False}